In [ ]:
import calendar
import os

import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

In [ ]:
path = '../data/tower_coords.csv'
tower_coords = pd.read_csv(path)
tower_coords

In [ ]:
mk_theilsen_results = pd.read_csv('../eda_results/mk_theilsen_results.csv')
mk_theilsen_results['tower'] = mk_theilsen_results['tower_variable_height'].apply(lambda x: x.split('_')[0])
mkts_coords = pd.merge(left=mk_theilsen_results, right=tower_coords, on='tower', how='left')
mkts_gdf = gpd.GeoDataFrame(mkts_coords, geometry=gpd.points_from_xy(x=mkts_coords['lon'], y=mkts_coords['lat'], crs='EPSG:4326'))
mkts_gdf

# Monthly Sens's slope per tower

In [ ]:
# Create a 'variable' & 'height' column by extracting from 'tower_variable_height'
mkts_gdf['variable'] = mkts_gdf['tower_variable_height'].apply(lambda x: '_'.join(x.split('_')[1:-1]))
mkts_gdf['height'] = mkts_gdf['tower_variable_height'].apply(lambda x: int(x.split('_')[-1][:-1]))  # Extract numeric height

# Define a base vertical separation factor
vertical_sep_factor = 0.0035  # Increase this factor for more vertical separation

# Define map bounds
lon_min_global, lat_min_global, lon_max_global, lat_max_global = mkts_gdf.total_bounds

# Iterate over each unique combination of variable and month
for (variable, month), group in mkts_gdf.groupby(['variable', 'month']):
    
    fig, ax = plt.subplots(figsize=(10, 10))

    # Normalize the Sens slope values to map to colors
    norm = Normalize(vmin=-0.001, vmax=0.001)
    cmap = plt.get_cmap('jet')  # Use a diverging colormap

    # Create a color bar based on Sens slope values
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Necessary to create a colorbar
    fig.colorbar(sm, ax=ax, label='Sens Slope')

    # Calculate bubble sizes based on the absolute value of Sens slope
    bubble_sizes = np.abs(group['sens_slope']) * 1000000  # Bubble size proportional to slope

    # Group by tower within each variable-month group
    for tower, tower_group in group.groupby('tower'):
        # Sort by height (ascending) for each tower
        tower_group = tower_group.sort_values(by='height')

        # Initialize a cumulative y-offset tracker
        cumulative_y_offset = 0.0
        
        # Plot each point with consistent vertical separation
        for idx, (x, y, size, label, height, sens_slope) in enumerate(zip(tower_group['lon'], tower_group['lat'], 
                                                                         bubble_sizes.loc[tower_group.index], 
                                                                         tower_group['tower'], tower_group['height'], tower_group['sens_slope'])):
            radius = np.sqrt(size / np.pi) * 0.00001  # Calculate radius from bubble size

            # Apply cumulative y-offset based on the bubble size
            if idx == 0:
                jittered_y = y  # No offset for the first bubble
            else:
                # Apply vertical separation based on previous bubble's radius
                cumulative_y_offset += (radius * 2.5) + vertical_sep_factor
                jittered_y = y + cumulative_y_offset

            # Plot the bubble at the (possibly jittered) y-coordinate, color by Sens slope
            ax.scatter(x, jittered_y, s=size, color=cmap(norm(sens_slope)), alpha=0.7, zorder=2)

            # Plot black center dots to differentiate
            ax.scatter(x, jittered_y, s=5, c='black', zorder=3)

            # Add a text offset separately from the bubble size
            text_offset = radius * 30  # Ensure sufficient space between the bubble and the label
            ax.text(x + text_offset, jittered_y, f"{label} ({height}m)", fontsize=11, ha='left', va='center', color='black', zorder=5)

    # Adjust x and y limits with a fixed zoom-out factor for consistent padding
    lon_min, lat_min, lon_max, lat_max = group.total_bounds
    zoom_out_factor = 0.003

    # Set the same x and y limits for every plot based on global min and max
    ax.set_xlim([lon_min_global - 0.01, lon_max_global + 0.02])  # Add extra space for text
    ax.set_ylim([lat_min_global - 0.01, lat_max_global + 0.01])  # Add extra space for bubbles

    # Add basemap as the bottom layer (zorder 1)
    ctx.add_basemap(ax, crs=group.crs.to_string(), source=ctx.providers.Esri.WorldTopoMap, zorder=1, attribution=False, reset_extent=True, zoom=15)
    
    # Convert month number to month name
    month_name = calendar.month_name[month]

    # Hide axis labels
    ax.set_axis_off()

    # Save or display the plot
    fig.title(f'{variable} {month_name}')
    
    # save figure with bbox_inches='tight' to ensure everything, including the legend, is included
    save_path = f'../graphics/maps/{variable}_{month_name}_sens.png'
    directory = os.path.dirname(save_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    print(f'Saving image to {save_path}')
    
    # Save figure with bbox_inches='tight'
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)
    # plt.savefig(f'{variable}_{month_name}_bubble.png', bbox_inches='tight')  # Uncomment to save each plot

In [ ]:
unique_geometry_df = mkts_gdf.drop_duplicates(subset='geometry', keep='first')
unique_geometry_df['tower'] = unique_geometry_df['tower_variable_height'].apply(lambda x: x.split('_')[0])
geom_df = unique_geometry_df[['tower', 'geometry']]

# Tower locations

In [ ]:
# Define map bounds
lon_min_global, lat_min_global, lon_max_global, lat_max_global = geom_df.total_bounds

# Iterate over each unique combination of variable and month
ax = geom_df.plot(figsize=(10, 10), color='red', marker='^', markersize=50, )

# Add basemap as the bottom layer (zorder 1)
ctx.add_basemap(ax, crs=geom_df.crs.to_string(), source=ctx.providers.Esri.WorldTopoMap, attribution='')
for x, y, label in zip(geom_df.geometry.x, geom_df.geometry.y, geom_df.tower):
    ax.annotate(label, xy=(x, y + 0.002), va='top', ha='center')